In [1]:
import cobra
import os
import pandas as pd
import numpy as np
from cobra.core import Model, Gene, Metabolite, Reaction

print('cobrapy version:')
print(cobra.__version__)

# os.chdir("/Users/Mo/Desktop/lab/cobra_py_project/iPX1N/model_helper_fxns/")
# import helper_functions_ipber as hf

path = "/Users/maureencarey/local_documents/work/comparative_parasite_models/paradigm"
data_path = "/Users/maureencarey/local_documents/work/comparative_parasite_models/paradigm/data"
os.chdir(data_path+'/other_models')
iPfal17 = cobra.io.read_sbml_model("iPfal17.xml")
iPfal18 = cobra.io.load_json_model("iPfal18.json")
os.chdir(data_path)
# new_pf3D7 = cobra.io.load_json_model(".json")
# pb_ortho = cobra.io.load_json_model(".json")
# pb_no_ortho = cobra.io.load_json_model(".json")


cobrapy version:
0.13.4


FileNotFoundError: [Errno 2] No such file or directory: '/Users/maureencarey/local_documents/work/comparative_parasite_models/paradigm/data/other_models'

In [54]:
os.chdir(data_path)
met_document = pd.read_table('bigg_metabolites.txt')
rxn_document = pd.read_table('bigg_reactions.txt')
os.chdir("/Users/maureencarey/local_documents/work/comparative_parasite_models/paradigm/models/")
uni = cobra.io.load_json_model('universal_model_oct26_2018.json')
model2 = cobra.io.load_json_model('iPfal18.json')

/Users/maureencarey/.virtualenvs/cobrapy/lib/python3.6/site-packages/ipykernel_launcher.py:2 FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/Users/maureencarey/.virtualenvs/cobrapy/lib/python3.6/site-packages/ipykernel_launcher.py:3 FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


In [50]:
model = cobra.io.read_sbml_model('iPfal17.xml')

In [65]:
model.reactions.MLTHFt
model2.reactions.MLTHFte3

# model2.reactions.MLTHFtap
# model.reactions.MLTHFtap
# model2.reactions.MLTHFtmt
# model.reactions.MLTHFtmt

Reaction identifier,MLTHFt
Name,MLTHFt
Memory address,0x012efb6f98
Stoichiometry,mlthf_e <=> mlthf_c mlthf[e] <=> 5_10_Methylenetetrahydrofolate
GPR,(MAL8P1_13) or (PF11_0172)
Lower bound,-10.0
Upper bound,10.0


In [70]:
[rn.reaction for rn in uni.reactions if '>' not in rn.reaction]

[]

In [67]:
model2.reactions.MLTHFtap.lower_bound = -1000.
model2.reactions.MLTHFte3.lower_bound = -1000.
model2.reactions.MLTHFtmt.lower_bound = -1000.
model2.reactions.MLTHFtap.upper_bound = 1000.
model2.reactions.MLTHFte3.upper_bound = 1000.
model2.reactions.MLTHFtmt.upper_bound = 1000.
model2.slim_optimize()

29.867730768742703

In [61]:
[r.id for r in model2.reactions if r.id.startswith('MLT')]

['MLTHFtap', 'MLTHFtmt', 'MLTHFte3']

In [22]:
iPfal17.id = 'iPfal17'
iPfal17.name = 'iPfal17'
iPfal18.id = 'iPfal18'
iPfal18.name = 'iPfal18'
# new_pf3D7.id = 'new_pf3D7'
# new_pf3D7.name = 'new_pf3D7'
# pb_ortho.id = 'pbo'
# pb_ortho.id = 'P. berghei, with orthology'
# pb_no_ortho.id = 'pbno'
# pb_no_ortho.name = 'P. berghei, without orthology'


In [31]:
print(len(iPfal18.reactions))
cobra.manipulation.modify.convert_to_irreversible(iPfal18)
print(len(iPfal18.reactions))
iPfal18.reactions[0].flux_expression

1838
1838


-1.0*ARGN_reverse_8a0ee + 1.0*ARGN

In [45]:
os.chdir(data_path)

# update gene identifiers
aliases = pd.read_csv('Pfalciparum3D7_GeneAliases.csv')
rename_dictionary1 = dict()
for index, row in aliases.iterrows():
    for x in ['name2','name3','name4','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7']:
        if row[x] != 'nan':
            rename_dictionary1[row[x]] = row['name1']
    
rename_dictionary = dict()
for key, value in rename_dictionary1.items():
    if isinstance(key, str):
        if key in [x.id for x in iPfal17.genes]:
            rename_dictionary[key] = value
        elif key.startswith('MAL'):
            s = key.replace('.','_')
            if s in [x.id for x in iPfal17.genes]:
                rename_dictionary[s] = value
                
for key, value in rename_dictionary.items():
    if '.' in value:
        rename_dictionary[key] = value.split('.')[0]

def rename_genes_updated(cobra_model, rename_dict):

    from six import iteritems
    from ast import NodeTransformer
    from cobra.core import Gene, Metabolite, Reaction
    from cobra.core.gene import ast2str
    from cobra.manipulation.delete import get_compiled_gene_reaction_rules

    recompute_reactions = set()  # need to recomptue related genes
    remove_genes = []
    for old_name, new_name in iteritems(rename_dict):
        # undefined if there a value matches a different key
        # because dict is unordered
        try:
            gene_index = cobra_model.genes.index(old_name)
        except ValueError:
            gene_index = None
        old_gene_present = gene_index is not None
        new_gene_present = new_name in cobra_model.genes
        if old_gene_present and new_gene_present:
            old_gene = cobra_model.genes.get_by_id(old_name)
            if old_gene != cobra_model.genes.get_by_id(new_name): # Added in case not renaming some
                remove_genes.append(old_gene)
                recompute_reactions.update(old_gene._reaction)
        elif old_gene_present and not new_gene_present:
            # rename old gene to new gene
    #         old_gene = cobra_model.genes.get_by_id(old_name) #added
            gene = cobra_model.genes[gene_index]
            # trick DictList into updating index
            cobra_model.genes._dict.pop(gene.id)  # ugh
            gene.id = new_name
            cobra_model.genes[gene_index] = gene
    #         recompute_reactions.update(old_gene._reaction) # ADDED
        elif not old_gene_present and new_gene_present:
            pass # already fixed
        else:  # not old gene_present and not new_gene_present
            # the new gene's _model will be set by repair
            # cobra_model.genes.append(Gene(new_name)) # Removed, otherwise, adds genes that are unassigned to reactions
            pass

    cobra_model.repair()

    class Renamer(NodeTransformer):
        def visit_Name(self, node):
            node.id = rename_dict.get(node.id, node.id)
            return node

    gene_renamer = Renamer()
    for rxn, rule in iteritems(get_compiled_gene_reaction_rules(cobra_model)):
        if rule is not None:
            rxn._gene_reaction_rule = ast2str(gene_renamer.visit(rule))

    for rxn in recompute_reactions:
        rxn.gene_reaction_rule = rxn._gene_reaction_rule
    for i in remove_genes:
        cobra_model.genes.remove(i)
    
rename_genes_updated(iPfal17,rename_dictionary)

In [51]:
iPfal17.reactions.TKT1.gene_reaction_rule = 'PF3D7_0610800'
iPfal17.reactions.TKT2.gene_reaction_rule = 'PF3D7_0610800'
iPfal18.reactions.TKT1.gene_reaction_rule = 'PF3D7_0610800'
iPfal18.reactions.TKT2.gene_reaction_rule = 'PF3D7_0610800'
# why didn't this get fixed
cobra.manipulation.delete.remove_genes(iPfal17,['PFF0530w'])
cobra.manipulation.delete.remove_genes(iPfal18,['PFF0530w'])
iPfal18.reactions.TKT2

Reaction identifier,TKT2
Name,TKT2
Memory address,0x0120d7da90
Stoichiometry,e4p_c + xu5p__D_c <=> f6p_c + g3p_c D_Erythrose_4_phosphate + D_Xylulose_5_phosphate <=> D_Fructose_6_phosphate + Glyceraldehyde_3_phosphate
GPR,PF3D7_0610800
Lower bound,-1000.0
Upper bound,1000.0


In [54]:
# simulate knockouts
simulations = dict()
for model in [iPfal17, iPfal18]: #,new_pf3D7, pb_ortho, pb_no_ortho]:
    model.solver = 'gurobi'
    
    deletion_results = cobra.flux_analysis.single_gene_deletion(model)
    no_grow = deletion_results.loc[deletion_results['growth'] < 0.1]
    grow = deletion_results.loc[deletion_results['growth'] >= 0.1]
    simulations[model.id] = {'essential': no_grow, 'dispensible' : grow}
    
    
    for y in grow.index:
        for x in list(y):
            if x.endswith('.1') or  x.endswith('.2') or  x.endswith('.3'):
                print(x)
    for y in no_grow.index:
        for x in list(y):
            if x.endswith('.1') or  x.endswith('.2') or  x.endswith('.3'):
                print(x)
    if len(grow) + len(no_grow.index) != len(model.genes):
        print('wrong number of genes')

In [78]:
# read experimental data
os.chdir(data_path) # read plenoplasm data in PF
data_pf = pd.read_csv('essentiality_file_ZhangTableS5.csv')

headers = list(data_pf)
headers.append("simulation")
data_pf = data_pf.reindex(columns = headers) 

# The higher the MIS value, the higher the probability of pathway dispensability.
# According to the paper, the 2000 genes with highest and lowest MISs represent
# mutability characterization with the strongest confidence, thus:
# get the indices of 2000 genes with lowest MISs
low_MIS_indices = data_pf.MIS.sort_values(ascending=True)[0:2000].index
low_MIS_genes = data_pf.Gene_ID[low_MIS_indices]
nonMutable = low_MIS_genes # a series
nM = list()
for x in nonMutable:
    if '.' in x: nM.append(x.split('.')[0])
    else: nM.append(x)
nonMutable_pf = nM

data_pb = pd.read_csv('essentiality_file_BushellTableS1.csv')
is_essential = data_pb['Phenotype']=='Essential'
nonMutable = data_pb[is_essential]['P. berghei current ID']
for x in nonMutable:
    if isinstance(x, float):
        print(x)
    else:
        if ";" in x:
            for y in x.split(';'):
                nM.append(y)
        else: nM.append(x)
nonMutable_pb = nM 

# toxoplasma gondii GT1
data_tg = pd.read_csv('essentiality_file_SidikTableS3_phenotype.csv')
# is_essential = data_pb['Phenotype']=='Essential'
# filter by all >-2 Non Essential
# filter by all <-2 Essential
is_essential = list()
for index, row in data_tg.iterrows():
    if -2> row['mean phenotype'] and -2 > max([x for x in row if not isinstance(x, str)]):
        is_essential.append(row['ID'])
nonMutable_pb = set(is_essential)

# Trypanosoma brucei 2T1
data_tb2T1 = pd.read_csv('essentiality_file_StortzTableXX.csv')
# filter by Tm/T <.5 = Essential

# Trypanosoma brucei MiTat 1.2, clone 221a UNDER DRUG PRESSURE

# Trypanosoma brucei UNDER DRUG PRESSURE
# Baker Molec and Biochemical Parasitology 

# Trypanosoma brucei KINOME RNAi, in vitro and in vivo
# Fernandez-Cortes Sci Reports 2017
# essential = induced/uninduced ratios < 0.6

nan
True
True
True


8158

In [58]:
comparison = dict()
for model_name, sim_results in simulations.items():
    
    if model_name in ['iPfal17','iPfal18','new_pf3D7']:
        print(model_name+' is pf'')
        nonMutable = nonMutable_pf
    else:
        print(model_name+' is pb')
        nonMutable = nonMutable_pb
    results = dict()
    no_grow = sim_results['essential'].index.tolist()
    list_genes = [item for sublist in [list(x) for x in no_grow] for item in sublist]
    for x in list_genes:
        if x in nonMutable:
            results[x] = 'nonMutable'
        else:
            results[x] = 'false positive'
    grow = sim_results['dispensible'].index.tolist()
    list_genes = [item for sublist in [list(x) for x in grow] for item in sublist]
    for x in list_genes:
        if x in nonMutable:
            results[x] = 'false negative'
        else:
            results[x] = 'Mutable'
     
    comparison[model_name] = results

{'iPfal17': {'PF3D7_0102900': 'false negative',
  'PF3D7_0103200': 'false negative',
  'PF3D7_0103700': 'Mutable',
  'PF3D7_0104400': 'false positive',
  'PF3D7_0106900': 'nonMutable',
  'PF3D7_0109500': 'Mutable',
  'PF3D7_0109800': 'false negative',
  'PF3D7_0109900': 'Mutable',
  'PF3D7_0111500': 'nonMutable',
  'PF3D7_0112200': 'Mutable',
  'PF3D7_0202700': 'nonMutable',
  'PF3D7_0203500': 'Mutable',
  'PF3D7_0204500': 'Mutable',
  'PF3D7_0204700': 'false positive',
  'PF3D7_0204900': 'Mutable',
  'PF3D7_0206300': 'false positive',
  'PF3D7_0206700': 'nonMutable',
  'PF3D7_0208500': 'Mutable',
  'PF3D7_0209100': 'Mutable',
  'PF3D7_0209300': 'nonMutable',
  'PF3D7_0209600': 'false negative',
  'PF3D7_0210300': 'false negative',
  'PF3D7_0211300': 'nonMutable',
  'PF3D7_0211400': 'Mutable',
  'PF3D7_0211600': 'false negative',
  'PF3D7_0211800': 'false negative',
  'PF3D7_0215000': 'Mutable',
  'PF3D7_0215300': 'Mutable',
  'PF3D7_0217100': 'false negative',
  'PF3D7_0219100': 'Muta

In [62]:
final_results = dict()
for model_name, comparison_results in comparison.items():
    true_pos = 0
    false_pos = 0
    false_neg = 0
    true_neg = 0
    for gene, result in comparison_results.items():
        if result == 'nonMutable':
            true_pos = true_pos + 1
        if result == 'false positive':
            false_pos = false_pos + 1
        if result == 'false negative':
            false_neg = false_neg + 1
        if result == 'Mutable':
            true_neg = true_neg + 1
    final_results[model_name] = {'accuracy': ((true_pos+true_neg)/(true_pos+true_neg+false_pos+false_neg)),
                                'false_pos_rate':(false_pos/(false_pos+true_neg)),
                                'false_neg_rate':(false_neg/(false_neg+true_pos)),
                                'genes_tested':(true_pos+true_neg+false_pos+false_neg)}
final_results

{'iPfal17': {'accuracy': 0.5936842105263158,
  'false_neg_rate': 0.7329842931937173,
  'false_pos_rate': 0.18661971830985916,
  'genes_tested': 475},
 'iPfal18': {'accuracy': 0.5915789473684211,
  'false_neg_rate': 0.7382198952879581,
  'false_pos_rate': 0.18661971830985916,
  'genes_tested': 475}}